In [1]:
import sys
sys.path.append('../modules')

from model import Resnet, Bottleneck

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from pathlib import Path 
import numpy as np

In [2]:
# Param

batch_size = 128

In [3]:
train_mean = (0.4914, 0.4822, 0.4465)
train_std = (0.2470, 0.2435, 0.2616)

In [4]:
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(train_mean, train_std)])

In [5]:
testset = torchvision.datasets.CIFAR10(root='./data/cifar10', train=False, download=False, transform=transform_test)
test_dataloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

model_save_path = './model'
model_list = sorted([x for x in Path(model_save_path).glob('*_epoch_*.pt')], key = lambda x : int(x.stem.strip('model_epoch_')))

best_model = model_list[190]

In [8]:
hit = 0
all_preds = []
all_labels = []

net = Resnet(Bottleneck, [3, 4, 6, 3]).to(device)
net = torch.nn.DataParallel(net, device_ids=[3,4,6,7]).to(device)
net.load_state_dict(torch.load(best_model)['state'])

criterion = nn.CrossEntropyLoss()

net.eval()

with torch.no_grad():
    test_loss = 0
    for (test_x, test_y) in tqdm(test_dataloader):
        preds = net(test_x.to(device))
        loss = criterion(preds, test_y.to(device))
        test_loss += loss
        hit += ((preds.argmax(dim=1) == test_y.to(device)).sum().item())

        all_preds.append(preds.argmax(dim=1))
        all_labels.append(test_y)

    accuracy = 100 * (hit/len(testset))
    print(f'accuracy : {accuracy:.4f}')
    print(f'test loss : {test_loss}')

    del net

100%|██████████| 79/79 [00:06<00:00, 11.35it/s]

accuracy : 90.1100
test loss : 48.911556243896484
